# ChatClovaX

This notebook provides a quick overview for getting started with Naver’s HyperCLOVA X [chat models](https://python.langchain.com/v0.2/docs/concepts/#chat-models) via CLOVA Studio. For detailed documentation of all ChatClovaX features and configurations head to the [API reference](https://api.python.langchain.com/en/latest/chat_models/langchain_community.chat_models.naver.ChatClovaX.html).

CLOVA Studio has several chat models. You can find information about latest models and their costs, context windows, and supported input types in the CLOVA Studio API Guide [documentation](https://api.ncloud-docs.com/docs/clovastudio-chatcompletions).

## Overview
### Integration details

| Class | Package | Local | Serializable | [JS support](https://js.langchain.com/v0.2/docs/integrations/chat/naver) | Package downloads | Package latest |
| :--- | :--- |:-----:| :---: |:------------------------------------------------------------------------:| :---: | :---: |
| [ChatClovaX](https://api.python.langchain.com/en/latest/chat_models/langchain_community.chat_models.naver.ChatClovaX.html) | [langchain-community](https://api.python.langchain.com/en/latest/community_api_reference.html) |   ❌   | beta/❌ |                                    ❌                                     | ![PyPI - Downloads](https://img.shields.io/pypi/dm/langchain-naver?style=flat-square&label=%20) | ![PyPI - Version](https://img.shields.io/pypi/v/langchain-naver?style=flat-square&label=%20) |

### Model features
| [Tool calling](/docs/how_to/tool_calling/) | [Structured output](/docs/how_to/structured_output/) | JSON mode | [Image input](/docs/how_to/multimodal_inputs/) | Audio input | Video input | [Token-level streaming](/docs/how_to/chat_streaming/) | Native async | [Token usage](/docs/how_to/chat_token_usage_tracking/) | [Logprobs](/docs/how_to/logprobs/) |
|:------------------------------------------:| :---: | :---: | :---: |  :---: | :---: |:-----------------------------------------------------:| :---: |:------------------------------------------------------:|:----------------------------------:|
|❌| ❌ | ❌ | ❌ | ❌ | ❌ |                          ✅                            | ✅ |                           ✅                            |                 ❌                  | 

## Setup

Before using the chat model, you must go through the three steps below.

1. Creating [NAVER Cloud Platform](https://www.ncloud.com/) account 
2. Apply to use [CLOVA Studio](https://www.ncloud.com/product/aiService/clovaStudio)
3. Find API Keys after creating CLOVA Studio Test App or Service App (See [here](https://guide.ncloud-docs.com/docs/en/clovastudio-playground01#테스트앱생성).)

### Credentials

CLOVA Studio requires 2 keys (`NCP_CLOVASTUDIO_API_KEY` and `NCP_APIGW_API_KEY`).
  - `NCP_CLOVASTUDIO_API_KEY` is issued per Test App or Service App
  - `NCP_APIGW_API_KEY` is issued per account

The two API Keys could be found by clicking `App Request Status` > `Service App, Test App List` > `‘Details’ button for each app` in [CLOVA Studio](https://clovastudio.ncloud.com/studio-application/service-app)

In [ ]:
import getpass
import os

os.environ["NCP_CLOVASTUDIO_API_KEY"] = getpass.getpass("NCP CLOVA Studio API Key: ")
os.environ["NCP_APIGW_API_KEY"] = getpass.getpass("NCP API Gateway API Key: ")

### Installation

The LangChain Naver integration lives in the `langchain-community` package:

In [ ]:
# install package
!pip install -qU langchain-community

## Instantiation

Now we can instantiate our model object and generate chat completions:

In [ ]:
from langchain_community.chat_models import ChatClovaX

chat = ChatClovaX(
    model="HCX-DASH-001",
    temperature=0.5,
    max_tokens=None,
    max_retries=2,
    # clovastudio_api_key="..."  # if you prefer to pass api key in directly instead of using env vars
    # task_id="..."  # if you want to use fine-tuned model
    # service_app=False (default)  # True if using service app
    # include_ai_filters=False (default) ## True if you want to detect inappropriate content
    # other params...
)

## Usage

`ChatClovaX` supports all `invoke`, `batch`, `stream` methods of [`ChatModel`](/docs/how_to#chat-models) including async APIs.

In [ ]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to Korean. Translate the user sentence.",
    ),
    ("human", "I love using NAVER AI."),
]

In [ ]:
#using invoke
chat.invoke(messages)

In [ ]:
#using batch
chat.batch([messages, messages])

In [ ]:
#using stream
for chunk in chat.stream(messages):
    print(chunk.content, end="", flush=True)

## Chaining

We can [chain](/docs/how_to/sequence/) our model with a prompt template like so:

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}. Translate the user sentence.",
        ),
        (
            "human", 
            "{input}"
        ),
    ]
)

chain = prompt | chat
chain.invoke(
    {
        "input_language": "English",
        "output_language": "Korean",
        "input": "I love using NAVER AI.",
    }
)

## Additional functionalities

### Fine-tuning

You can call fine-tuned CLOVA X models by passing in your corresponding `task_id` parameter. (You don’t need to specify the model_name parameter when calling fine-tuned model.)

You can check `task_id` from corresponding Test App or Service App details.

In [ ]:
fine_tuned_model = ChatClovaX(
    task_id='abcd123e',
    temperature=0.5,
)

fine_tuned_model.invoke(messages)

### Service App

When going live with production-level application using CLOVA Studio, you should apply for and use Service App. (See [here](https://guide.ncloud-docs.com/docs/en/clovastudio-playground01#서비스앱신청).)

For a Service App, a corresponding `NCP_CLOVASTUDIO_API_KEY` is issued and can only be called with the API Key.

In [ ]:
#### Update environment variables

os.environ["NCP_CLOVASTUDIO_API_KEY"] = getpass.getpass("NCP CLOVA Studio API Key for Service App: ")

In [ ]:
chat = ChatClovaX(
    service_app=True, # True if you want to use your service app, default value is False.
    # clovastudio_api_key="..."  # if you prefer to pass api key in directly instead of using env vars
    # apigw_api_key="..."  # if you prefer to pass gateway key in directly instead of using env vars
    model="HCX-DASH-001",
    temperature=0.5,
    max_tokens=None,
    max_retries=2,
    # other params...
)
ai_msg = chat.invoke(messages)

### AI Filter

AI Filter detects inappropriate output such as profanity from the test app (or service app included) created in Playground and informs the user. See [here](https://guide.ncloud-docs.com/docs/en/clovastudio-playground01#AIFilter) for details. 

In [ ]:
chat = ChatClovaX(
    model="HCX-DASH-001",
    temperature=0.5,
    max_tokens=None,
    max_retries=2,
    include_ai_filters=True, # True if you want to enabled ai filter
    # other params...
)

ai_msg = chat.invoke(messages)

In [ ]:
print(ai_msg.response_metadata['ai_filter'])

## API reference

For detailed documentation of all ChatClovaX features and configurations head to the API reference: https://api.python.langchain.com/en/latest/chat_models/langchain_community.chat_models.naver.ChatClovaX.html